In [17]:
from datetime import datetime
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import PyPDF2
import io
import tempfile
import re
import time

class DODFMonitor:
    def __init__(self, email_remetente="", senha_email="", email_destinatario=""):
        self.email_remetente = email_remetente
        self.senha_email = senha_email
        self.email_destinatario = email_destinatario
        
        # Configuração do Chrome
        self.chrome_options = Options()
        self.chrome_options.add_argument('--headless')
        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-dev-shm-usage')
        self.temp_dir = tempfile.mkdtemp()
        self.chrome_options.add_argument(f'--user-data-dir={self.temp_dir}')
        
        # Configuração da sessão HTTP com retry
        self.session = requests.Session()
        retries = Retry(
            total=5,  # número total de tentativas
            backoff_factor=1,  # tempo entre tentativas
            status_forcelist=[500, 502, 503, 504, 404],
            allowed_methods=["HEAD", "GET", "OPTIONS"]
        )
        adapter = HTTPAdapter(max_retries=retries)
        self.session.mount("http://", adapter)
        self.session.mount("https://", adapter)

    def baixar_pdf_com_retry(self, url, max_tentativas=3):
        """Tenta baixar o PDF com múltiplas tentativas"""
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0',
            'Accept': 'application/pdf,application/x-pdf',
            'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
            'Referer': 'https://dodf.df.gov.br/',
            'Connection': 'keep-alive'
        }

        for tentativa in range(max_tentativas):
            try:
                print(f"Tentativa {tentativa + 1} de {max_tentativas} para baixar o PDF...")
                
                response = self.session.get(
                    url,
                    headers=headers,
                    timeout=(30, 90),  # (connect timeout, read timeout)
                    stream=True
                )
                
                if response.status_code == 200:
                    # Verificar se é realmente um PDF
                    if 'application/pdf' in response.headers.get('content-type', '').lower():
                        return io.BytesIO(response.content)
                    else:
                        print("O conteúdo retornado não é um PDF")
                else:
                    print(f"Status code: {response.status_code}")
                
                if tentativa < max_tentativas - 1:
                    tempo_espera = (tentativa + 1) * 5  # Espera progressiva
                    print(f"Aguardando {tempo_espera} segundos antes da próxima tentativa...")
                    time.sleep(tempo_espera)
                    
            except Exception as e:
                print(f"Erro na tentativa {tentativa + 1}: {str(e)}")
                if tentativa < max_tentativas - 1:
                    tempo_espera = (tentativa + 1) * 5
                    print(f"Aguardando {tempo_espera} segundos antes da próxima tentativa...")
                    time.sleep(tempo_espera)
        
        return None

    def gerar_link_pasta_dodf(self, data):
        """Gera o link da pasta do DODF para uma data específica"""
        data_obj = datetime.strptime(data, '%d/%m/%Y')
        
        meses = {
            1: 'Janeiro', 2: 'Fevereiro', 3: 'Marco', 4: 'Abril',
            5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
            9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
        }
        
        ano = data_obj.strftime('%Y')
        mes_numero = data_obj.month
        mes_nome = meses[mes_numero]
        data_formatada = f"{data_obj.strftime('%d')} {data_obj.strftime('%m')} {data_obj.strftime('%Y')}"
        
        return f"https://dodf.df.gov.br/dodf/jornal/pastas?pasta={ano}/{mes_numero:02d}_{mes_nome}/{data_formatada}".replace(" ", "%20")

    def obter_edicoes_do_dia(self, data):
        """Obtém informações sobre as edições disponíveis para uma data"""
        url = self.gerar_link_pasta_dodf(data)
        driver = None
        
        try:
            driver = webdriver.Chrome(options=self.chrome_options)
            print(f"Acessando pasta do dia {data}...")
            driver.get(url)
            
            wait = WebDriverWait(driver, 20)
            elementos = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "link-materia")))
            
            edicoes = []
            for elemento in elementos:
                nome_arquivo = elemento.find_element(By.TAG_NAME, "p").text.strip()
                link = elemento.get_attribute("href")
                
                # Extrai o número da edição do nome do arquivo
                # Formato esperado: "DODF XXX DD-MM-AAAA ..."
                match = re.search(r'DODF (\d+)', nome_arquivo)
                if match:
                    numero_edicao = match.group(1)
                    edicoes.append({
                        'numero': numero_edicao,
                        'nome': nome_arquivo,
                        'link': link
                    })
                    print(f"Encontrada edição {numero_edicao}")
            
            return edicoes
            
        except Exception as e:
            print(f"Erro ao acessar pasta: {e}")
            return None
        finally:
            if driver:
                driver.quit()

    def baixar_e_pesquisar_pdf(self, url, nome_busca):
        """Baixa e pesquisa um nome no PDF"""
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Referer': 'https://dodf.df.gov.br/'
        }
        
        try:
            print(f"Baixando PDF: {url}")
            response = requests.get(url, headers=headers, timeout=30)
            
            if response.status_code == 200:
                pdf_buffer = io.BytesIO(response.content)
                return self.pesquisar_nome_pdf(pdf_buffer, nome_busca)
            else:
                print(f"Erro ao baixar PDF. Status code: {response.status_code}")
                return None
                
        except Exception as e:
            print(f"Erro ao baixar ou processar PDF: {e}")
            return None

    def pesquisar_nome_pdf(self, pdf_buffer, nome):
        """Pesquisa um nome no PDF e retorna os contextos encontrados"""
        resultados = []
        
        try:
            pdf_reader = PyPDF2.PdfReader(pdf_buffer)
            
            for num_pagina, pagina in enumerate(pdf_reader.pages, 1):
                texto = pagina.extract_text()
                texto_lower = texto.lower()
                nome_lower = nome.lower()
                
                if nome_lower in texto_lower:
                    indice = texto_lower.find(nome_lower)
                    inicio = max(0, indice - 200)
                    fim = min(len(texto), indice + len(nome) + 200)
                    contexto = texto[inicio:fim]
                    
                    resultados.append({
                        'pagina': num_pagina,
                        'contexto': contexto.strip()
                    })
            
            return resultados
            
        except Exception as e:
            print(f"Erro ao processar PDF: {e}")
            return None

    def pesquisar_nome_por_data(self, nome, data):
        """Pesquisa um nome em todas as edições de uma data específica"""
        print(f"\nPesquisando '{nome}' na data {data}")
        
        try:
            # Primeiro obtém as edições do dia
            edicoes = self.obter_edicoes_do_dia(data)
            
            if not edicoes:
                print("Nenhuma edição encontrada para esta data")
                return
            
            resultados_totais = []
            
            # Para cada edição encontrada
            for edicao in edicoes:
                print(f"\nVerificando edição {edicao['numero']}...")
                
                # Tenta baixar e pesquisar o PDF com retry
                pdf_buffer = self.baixar_pdf_com_retry(edicao['link'])
                
                if pdf_buffer:
                    resultados = self.pesquisar_nome_pdf(pdf_buffer, nome)
                    if resultados:
                        resultados_totais.append({
                            'edicao': edicao['nome'],
                            'link': edicao['link'],
                            'mencoes': resultados
                        })
                else:
                    print(f"Não foi possível baixar o PDF da edição {edicao['numero']}")
            
            return resultados_totais
            
        except Exception as e:
            print(f"Erro durante a pesquisa: {str(e)}")
            return None
# Exemplo de uso


In [18]:
# Exemplo de uso
if __name__ == "__main__":
    monitor = DODFMonitor()
    
    # Data e nome para pesquisar
    data = "09/08/2023"
    nome = "Henrique do Vale Rocha Filho"
    
    # Faz a pesquisa
    resultados = monitor.pesquisar_nome_por_data(nome, data)
    
    # Exibe os resultados
    if resultados:
        print(f"\nResultados encontrados para '{nome}' em {data}:")
        for resultado in resultados:
            print(f"\nEdição: {resultado['edicao']}")
            print(f"Link: {resultado['link']}")
            print("\nMenções encontradas:")
            
            for mencao in resultado['mencoes']:
                print(f"\nPágina {mencao['pagina']}:")
                print(f"Contexto: ...{mencao['contexto']}...")
                print("-" * 80)
    else:
        print(f"\nNenhuma menção a '{nome}' encontrada na data {data}")


Pesquisando 'Henrique do Vale Rocha Filho' na data 09/08/2023


Cache folder (/Users/henrique/.cache/selenium) cannot be created: Permission denied (os error 13)
Cache folder (/Users/henrique/.cache/selenium) cannot be created: Permission denied (os error 13)
Cache folder (/Users/henrique/.cache/selenium) cannot be created: Permission denied (os error 13)


Acessando pasta do dia 09/08/2023...


/var/folders/fj/821tv6qn54xbnrgw2tjk7jf00000gn/T/ipykernel_60221/3601343212.py:119: DeprecationWarning: using WebElement.get_attribute() has been deprecated. Please use get_dom_attribute() instead.
  link = elemento.get_attribute("href")


Encontrada edição 150

Verificando edição 150...
Tentativa 1 de 3 para baixar o PDF...
Erro na tentativa 1: HTTPSConnectionPool(host='dodf.df.gov.br', port=443): Max retries exceeded with url: /dodf/jornal/visualizar-pdf?pasta=2023%7C08_Agosto%7CDODF%20150%2009-08-2023%7C&arquivo=DODF%20150%2009-08-2023%20INTEGRA.pdf (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x106baa840>, 'Connection to dodf.df.gov.br timed out. (connect timeout=30)'))
Aguardando 5 segundos antes da próxima tentativa...
Tentativa 2 de 3 para baixar o PDF...
Erro na tentativa 2: HTTPSConnectionPool(host='dodf.df.gov.br', port=443): Max retries exceeded with url: /dodf/jornal/visualizar-pdf?pasta=2023%7C08_Agosto%7CDODF%20150%2009-08-2023%7C&arquivo=DODF%20150%2009-08-2023%20INTEGRA.pdf (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x106bab6b0>, 'Connection to dodf.df.gov.br timed out. (connect timeout=30)'))
Aguardando 10 segundos antes da próxima tentati

In [8]:
# Exemplo de uso
if __name__ == "__main__":
    # Configurações do email
    EMAIL_REMETENTE = "henriquef29@gmail.com    "
    SENHA_EMAIL = "IquinhoF27@"  # Para Gmail, usar senha de aplicativo
    EMAIL_DESTINATARIO = "henriquevrocha@me.com"
    
    # Criar monitor
    monitor = DODFMonitor(EMAIL_REMETENTE, SENHA_EMAIL, EMAIL_DESTINATARIO)
    
    # Monitorar um nome
    nome_procurado = "Henrique do Vale Rocha Filho"
    monitor.monitorar_nome(nome_procurado)

Verificando menções a 'Henrique do Vale Rocha Filho' na data 09/09/2023


Cache folder (/Users/henrique/.cache/selenium) cannot be created: Permission denied (os error 13)
Cache folder (/Users/henrique/.cache/selenium) cannot be created: Permission denied (os error 13)
Cache folder (/Users/henrique/.cache/selenium) cannot be created: Permission denied (os error 13)


Erro ao obter edições: Message: 

Nenhuma edição encontrada para esta data.
